# 1. 로그인 (log in)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import configparser, time,requests,os,re
from datetime import datetime
from dateutil import parser  # dateutil.parser 모듈을 사용하여 날짜 문자열을 처리

# Instagram 로그인 정보
config = configparser.ConfigParser()
config.read("config.ini")

USERNAME = config["instagram"]["username"]
PASSWORD = config["instagram"]["password"]

# 브라우저 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
# options.add_argument("--headless")  # 헤드리스 모드 활성화
# options.add_argument("--window-size=1920,1080")  # 창 크기 설정 (선택 사항)
options.add_argument("--no-sandbox")  # 권한 관련 문제 방지 (선택 사항)
driver = webdriver.Chrome(options=options)

# WebDriverWait 객체 생성
wait = WebDriverWait(driver, 20)  # 최대 20초 대기

# Instagram 로그인 페이지로 이동
driver.get("https://www.instagram.com/accounts/login/")

# 로그인 입력 필드 대기 후 찾기
username_input = wait.until(EC.presence_of_element_located((By.NAME, "username")))
password_input = wait.until(EC.presence_of_element_located((By.NAME, "password")))

# 계정 정보 입력
username_input.send_keys(USERNAME)
password_input.send_keys(PASSWORD)
password_input.send_keys(Keys.RETURN)
time.sleep(5)

### 2. 다운로드 (find & download)

In [ ]:
config = configparser.ConfigParser()
config.read("config.ini")

ACCOUNTS_LIST = config["instagram"]["accounts_list"].splitlines()
ACCOUNTS_LIST = [url.strip() for url in ACCOUNTS_LIST if url and url.strip()]

# ☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★


def get_accounts_num(target_account, config):

    # 정규식으로 계정명과 숫자 추출
    match = re.match(r"(https?://www\.instagram\.com/)?([\w\.]+)(?:\s*//\s*(\d+))?", target_account)

    target_account = match.group(2)  # 계정명
    target_post_count = int(match.group(3)) if match.group(3) else int(config["instagram"]["DEFAULT_NUM"])

    return target_account, target_post_count


for target_account in ACCOUNTS_LIST:
    target_account, target_post_count = get_accounts_num(ACCOUNTS_LIST[1], config)

    driver.get(f"https://www.instagram.com/{target_account}/")

    # 특정 요소(예: 이미지 또는 게시물) 대기 후 찾기
    profile_loaded = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "img")))
    time.sleep(1)

    profile_name = driver.find_element(
        By.XPATH,
        "//span[@class='x1lliihq x1plvlek xryxfnj x1n2onr6 x1ji0vk5 x18bv5gf x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye xvs91rp x1s688f x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj']",
    ).text

    # 출력
    print(f"\n{profile_name} (insta_{target_account}) \t {0}/{target_post_count} - {0/target_post_count*100:.1f}%", end="")

    # 수집할 게시글 수 설정
    post_links = []  # 게시글 링크를 저장할 리스트

    last_height = driver.execute_script("return document.body.scrollHeight")

    while len(post_links) < target_post_count:
        # 게시글 링크 요소 찾기
        links = driver.find_elements(By.TAG_NAME, "a")
        new_links = []  # 이번 스크롤에서 새로 발견된 링크를 담을 리스트

        for link in links:
            href = link.get_attribute("href")
            # print(href)
            if ("/p/" in href or "/reel/" in href) and href not in post_links and href not in new_links:
                new_links.append(href.replace("/reel/", "/p/"))

        # 새로 발견된 링크들을 post_links에 추가
        post_links.extend(new_links)

        # 목표 게시글 수에 도달하면 종료
        if len(post_links) >= target_post_count:
            break

        # 페이지 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 로딩 대기

        # 새로운 높이 확인
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 스크롤 끝에 도달
            break
        last_height = new_height
    post_links = post_links[:target_post_count]
    # 결과 출력
    # print(f"총 {len(post_links)}개의 게시글 링크를 수집했습니다.")
    # ☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★
    # ☆★☆★☆★☆★☆★☆★☆★ 이미지 다운로드☆★☆★☆★☆★☆★☆★☆★☆★☆★
    # ☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆★

    # 이미지 저장 폴더 생성
    download_folder = f"{profile_name} (insta_{target_account})"

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    # 수집한 게시글 링크에서 이미지 다운로드 시작
    for article_index, post_url in enumerate(post_links):
        # print(f"{article_index}/{len(post_links)}: {post_url} 에서 이미지 다운로드 시작")

        # 게시글 링크에서 이미지 URL 추출하고 다운로드
        driver.get(post_url)
        time.sleep(2)  # 페이지 로딩 대기

        # 게시글 업로드 날짜 추출
        post_date = driver.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")
        # 'Z'를 처리할 수 있도록 dateutil.parser를 사용해 날짜 문자열 파싱
        post_date = parser.isoparse(post_date).strftime("%Y-%m-%d")

        # try:
        # 이미지 개수 확인을 위해 acnb 클래스의 개수를 세기
        acnb_elements = driver.find_elements(By.CSS_SELECTOR, "div._acnb")
        total_images = len(acnb_elements)
        if total_images <= 1:
            total_images = 1

        # print(f"총 {total_images}개의 사진/동영상 이 존재합니다.")

        # 이미지를 저장할 리스트와 중복을 체크할 set
        all_image_urls = []
        seen_image_urls = set()

        # 각 '다음 이미지' 클릭 후 이미지 URL 수집 (다운 X)
        for i in range(max(total_images, 1)):
            # 이미지 URL 수집
            # instagram.com/accounts/p/ 형태로 되있어야 나옴
            selector = "div._aatk._aatn" if total_images == 1 else "ul._acay"

            image_list = driver.find_element(By.CSS_SELECTOR, selector)
            if total_images > 1:
                image_list = image_list.find_elements(By.CSS_SELECTOR, "li._acaz")[0 if i == 0 else 1]  # [0 if i==0 else 1]

            images = image_list.find_elements(By.TAG_NAME, "img") #img or video

            if images:
                images = images[0]
                img_url = images.get_attribute("src")

                # 'scontent'가 URL에 포함된 이미지만 리스트에 추가
                if ("scontent" in img_url) and (img_url not in seen_image_urls):
                    all_image_urls.append(img_url)
                    seen_image_urls.add(img_url)
            else:
                video = image_list.find_element(By.TAG_NAME, "video")
                video_url = video.get_attribute("src")
                all_image_urls.append(video_url)

            # '다음 이미지' 버튼 클릭
            # if total_images >= 2:
            if (total_images >= 2) and (i < (total_images-1)):
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='다음']")
                    next_button.click()
                    time.sleep(1)  # 이미지 로딩 대기
                except:
                    pass

        # print(f"총 {len(all_image_urls)}개의 사진/동영상 URL 수집 완료")

        # 수집된 이미지 링크들을 이용해 다운로드 시작
        for idx, img_url in enumerate(all_image_urls):
            if 'blob:' in img_url: # video url 일 경우
                continue
            img_name = os.path.join(download_folder, f"[{post_date}] {post_url.rsplit('/', 2)[1]}_p{idx}.jpg")

            # 이미지 다운로드
            try:
                response = requests.get(img_url)
                if response.status_code == 200:
                    with open(img_name, "wb") as f:
                        f.write(response.content)
                    # print(f"이미지 {idx} 다운로드 완료: {img_name}")
                else:
                    # print(f"이미지 다운로드 실패: {img_url}")
                    pass
            except requests.exceptions.RequestException as e:
                print(f"이미지 다운로드 중 오류 발생: {e}")
        print(
            f"\r{profile_name} (insta_{target_account}) \t {article_index+1}/{target_post_count} - {(article_index+1)/target_post_count*100:.1f}%",
            end="",
        )

print("모든 이미지 다운로드 완료!")